In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote
import cProfile

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [297, 5606, 9938, 3602, 11291, 11858, 10597, 15224, 1307, 16021, 15727, 9986, 4649, 19728, 11451, 16468, 152, 15168, 14987, 13107, 2536, 10930, 9673, 19863, 19790, 11665, 12259, 4534, 10399, 17934, 12681, 8151, 622, 7859, 17132, 10349, 2420, 13165, 11894, 1658, 6236, 7015, 10961, 8721, 4586, 16274, 13416, 10564, 13015, 1292, 9342, 8159, 8308, 1632, 3986, 13132, 15419, 15262, 7908, 18209, 11517, 14304, 18499, 8657, 6228, 2455, 19813, 14016, 14297, 7902, 13810, 4003, 17541, 5525, 1538, 15076, 15669, 2034, 607, 14614, 13395, 2071, 2438, 15237, 10955, 1151, 14798, 14143, 17325, 18466, 10724, 12690, 14312, 15365, 575, 19291, 13302, 7412, 6769, 11747, 848, 14814, 17796, 13320, 950, 17149, 3045, 11422, 11838, 10352, 16139, 19888, 4537, 9860, 11067, 16090, 7550, 17250, 9029, 9740, 13691, 12577, 8436, 17071, 10834, 8884, 5316, 16803, 10526, 12008, 11076, 13340, 5856, 2512, 1368, 1730, 19004, 3042, 15791, 19234, 19984, 296, 14513, 5233, 19

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = torch.load('pre_decoder/decoder_a.pth')
decoder_p = torch.load('pre_decoder/decoder_p.pth')
decoder_v = torch.load('pre_decoder/decoder_v.pth')
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'noFT')
# profiler.disable()

Epoch [1/500], Loss: 1.3916, Accuracy: 0.2604, Edge Accuracy: [0.9997554988434233, 0.9996765616245255, 1.0]
Class 0: AUC-ROC- 0.4892, F1 Score- 0.0000; Class 1: AUC-ROC- 0.3683, F1 Score- 0.0000; Class 2: AUC-ROC- 0.3865, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4644, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.4271; Macro-Average F1 Score: 0.0000
tensor(1.3897, device='cuda:0')
Validation Loss: 1.3897, Validation Accuracy: 0.2500, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.5088, F1 Score- 0.0000; Class 1: AUC-ROC- 0.4180, F1 Score- 0.0000; Class 2: AUC-ROC- 0.3963, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4923, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.4538; Macro-Average F1 Score: 0.0000
tensor(1.3907, device='cuda:0')
Test Loss: 1.3907, Test Accuracy: 0.2500, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.4589, F1 Score- 0.0000; Class 1: AUC-ROC- 0.3241, F1 Score- 0.0000; Class 2: AUC-ROC- 0.3918, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4962, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.417

In [8]:
# profiler.print_stats(sort='cumulative')  # Print results to the console
# # Or save to a file
# with open('profile_results.txt', 'w') as f:
#     profiler.dump_stats(f)

In [9]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = 'noFT')

tensor(0.8829, device='cuda:0')
Test Loss: 0.8829, Test Accuracy: 0.9525, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.9996, F1 Score- 0.9802; Class 1: AUC-ROC- 0.9915, F1 Score- 0.9641; Class 2: AUC-ROC- 0.9936, F1 Score- 0.8995; Class 3: AUC-ROC- 0.9986, F1 Score- 0.9479; Macro-Average AUC-ROC: 0.9958; Macro-Average F1 Score: 0.9479


(0.9525, 0.9958166666666666, 0.9479223533958054)